In [1]:
import polars as pl
import pandas as pd 
import os
import re
import glob
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile
from rarfile import RarFile
from bs4 import BeautifulSoup
import difflib
from utils import guess_header, change_df_header, get_year_month_from_filename, treat_uf_tipo
DICT_UFS = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins",
}

In [2]:

#treat_uf_tipo('../files/2022/frota_por_uf_e_tipo_de_veículo_2-2022.xls')

In [8]:
from download_frota import download_frota

In [9]:
download_frota(2, 2022)

Download of frota_por_uf_e_tipo_de_veículo_2-2022.xls complete
Download of frota_por_município_e_tipo_2-2022.xls complete


In [3]:
import basedosdados as bd 
municipios_query = """SELECT nome, id_municipio, sigla_uf FROM `basedosdados.br_bd_diretorios_brasil.municipio`
"""
bd_municipios = bd.read_sql(municipios_query, 'tamir-pipelines')
bd_municipios = pl.from_pandas(bd_municipios)

/home/tamir/.cache/pypoetry/virtualenvs/pipelines-Fyk5jlG0-py3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.8) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 10744.58rows/s]


In [4]:
file = '../files/2022/frota_por_município_e_tipo_2-2022.xls'
filename = os.path.split(file)[1]
df = pd.read_excel(file)
new_df  = change_df_header(df, guess_header(df))
new_df.rename(columns={ new_df.columns[0]: "sigla_uf" , new_df.columns[1]: 'nome_denatran'}, inplace= True) # Rename for ease of use.
new_df.sigla_uf = new_df.sigla_uf.str.strip() # Remove whitespace.
new_df = pl.from_pandas(new_df)

In [5]:
uf = 'RN'

In [6]:
# for uf in DICT_UFS:
denatran_uf = new_df.filter(pl.col('sigla_uf') == uf)
ibge_uf = bd_municipios.filter(pl.col('sigla_uf') == uf)
ibge_uf = ibge_uf.with_columns(pl.col('nome').str.to_lowercase())

In [1]:
denatran_uf

NameError: name 'denatran_uf' is not defined

In [7]:
def get_city_name_ibge(denatran_name: str):
    matches = difflib.get_close_matches(denatran_name.lower(), ibge_uf['nome'].str.to_lowercase(), n = 1)
    if matches:
        return matches[0]
    else:
        return '' # I don't want this to error out directly, because then I can get all municipalities.
x = denatran_uf['nome_denatran'].apply(get_city_name_ibge)
denatran_uf = denatran_uf.with_columns(x.alias('suggested_nome_ibge'))

In [8]:
def match_ibge(denatran_uf: pl.DataFrame, ibge_uf: pl.DataFrame):
    joined_df = denatran_uf.join(ibge_uf, left_on=['suggested_nome_ibge', 'sigla_uf'], right_on=['nome', 'sigla_uf'], how='left')
    mismatched_rows = joined_df.filter(pl.col('id_municipio').is_null())

    if len(mismatched_rows) > 0:
        error_message = "Os seguintes municípios falharam: \n"
        for row in mismatched_rows.rows(named = True):
            error_message += f"{row['nome_denatran']} ({row['sigla_uf']})\n"
        raise ValueError(error_message)


In [9]:
match_ibge(denatran_uf, ibge_uf)

ValueError: Os seguintes municípios falharam: 
ASSU (RN)


In [232]:
joined_df = denatran_uf.join(ibge_uf, left_on=['suggested_nome_ibge', 'sigla_uf'], right_on=['nome', 'sigla_uf'], how='left')

mismatched = joined_df.filter(pl.col('id_municipio').is_null())

In [146]:
def verify_total(df: pl.DataFrame):
    columns_for_total = df.select( pl.exclude("TOTAL")).select(pl.exclude([pl.Utf8]))
    calculated_total = columns_for_total.select( pl.fold(        acc=pl.lit(0), function=lambda acc, x: acc + x, exprs=pl.col("*")
    ).alias("calculated_total"))['calculated_total']

    mask = df['TOTAL'] == calculated_total
    if pl.sum(~mask) != 0:
        raise ValueError('A coluna de TOTAL da base original tem inconsistências e não soma tudo das demais colunas.')
 # Verifies that we can do drop total and do LONG. 
verify_total(new_df)

In [12]:
import polars as pl

# create a sample polars data frame
df = pl.DataFrame({
    'nome_denatran': ['A', 'B', 'C', 'D'],
    'sigla_uf': ['SP', 'MG', 'RJ', 'ES'],
    'id_municipio': [0, 0, 0, 0]
})

# set id_municipio to 1 when nome_denatran is 'B' and sigla_uf is 'MG'
mask1 = (df['nome_denatran'] == 'B') & (df['sigla_uf'] == 'MG')
df['id_municipio'] = df.apply(lambda x: 1 if mask1[x] else x['id_municipio'], predicate=mask1)

# set id_municipio to 23 when nome_denatran is 'C' and sigla_uf is 'RJ'
mask2 = (df['nome_denatran'] == 'C') & (df['sigla_uf'] == 'RJ')
df['id_municipio'] = df.apply(lambda x: 23 if mask2[x] else x['id_municipio'], predicate=mask2)

# print the updated data frame
print(df)


TypeError: DataFrame.apply() got an unexpected keyword argument 'predicate'